# Applied Data Science Capstone Project: Segmenting and Clustering Neighborhoods in Toronto-Problem 1

**To scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe**

In [11]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [12]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [13]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

This means that there are 3 columns per row

In [14]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


**Creating Pandas DataFrame** <br>
Each header is appended to a tuple along with an empty list.

In [15]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [16]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[287, 287, 287]

This shows that each of the 3 columns has exactly 287 rows

**Creating the pandas data frame**

In [17]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [18]:
# Access the top 5 rows of the data frame 
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


**Rearranging and renaming the columns**

In [28]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood


**Cleaning the messy string in the Borough column**

In [29]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood


**Dropping all cells with a borough that is Not assigned**

In [30]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,M5H,Downtown Toronto,Adelaide
1,M1S,Scarborough,Agincourt
2,M1V,Scarborough,Agincourt North
3,M9V,Etobicoke,Albion Gardens
4,M8W,Etobicoke,Alderwood
5,M3H,North York,Bathurst Manor
6,M5V,Downtown Toronto,Bathurst Quay
7,M2K,North York,Bayview Village
8,M9V,Etobicoke,Beaumond Heights
9,M5M,North York,Bedford Park


**Combining Neighbourhoods based on similar Postcode and Borough**

In [31]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,M1A,Not assigned
1,M1B,Scarborough,"Malvern ,Rouge"
2,M1C,Scarborough,"Highland Creek ,Port Union ,Rouge Hill"
3,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park"
8,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge"
9,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West"


In [32]:
#Removing any space in the start of the string
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [33]:
#Assigning Borough values to the Neighbourhood where vlaue is "Not assigned"
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [34]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood
120,M7A,Queen's Park,Queen's Park


In [35]:
# Check the shape of the data frame
df.shape

(180, 3)

In [36]:
#Save this file to a csv
df.to_csv(r'df_can.csv')